In this notebook, the goal is to perform parallelized model selection and then report final test set accuracy.

Structure of notebook:

* Load data
* Perform train-test split
* Define dictionary of hyperparameters
* Do cross-validation to select best hyperparameters
* Train final model
* Evaluate final model on test set, using bootstrapping to obtain confidence intervals

Load base packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

from scipy.stats import zscore, zmap
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, KFold
from itertools import product
from joblib import Parallel, delayed
from multiprocessing import cpu_count

This command lets you edit `.py` files and have the changed versions be accessed by Jupyter.

In [ ]:
%load_ext autoreload
%autoreload 2

Load cadre modeling package

In [ ]:
sys.path.insert(0, '../cadreModels/')
from classificationBinary import binaryCadreModel

In [ ]:
sns.set_style('darkgrid')

Load `breastcancer` and then extract observations, labels, and features. Note that we're turning the labels into a rank-2 array.

In [ ]:
breastcancer = load_breast_cancer()
X, Y, features = breastcancer['data'], np.expand_dims(breastcancer['target'], 1), breastcancer['feature_names']

Map `Y` values to -1 and +1 for hinge loss

In [ ]:
Y = 2 * Y - 1
pd.DataFrame(Y)[0].value_counts()

Perform a randomized train-test split

In [ ]:
Xtr, Xte, Ytr, Yte = train_test_split(X, Y, test_size=0.2, random_state=1515)

Dictionary of hyperparameters used for model selection. We're holding the sparsity parameters `alpha_d` and `alpha_W` fixed at their default values of 0.9.

In [ ]:
scm_params = {'M': np.array([2,3]), 'lambda_d': np.array([0.01, 0.1]), 'lambda_W': np.array([0.01, 0.1])}

3-fold cross-validation index generator

In [ ]:
kf = KFold(n_splits=3, random_state=1414)

Arguments to the SCM initialization function:

* `M` -- number of cadres
* `lambda_d` -- regularization strength hyperparameter for cadre-assignment weight `d`
* `lambda_W` -- regularization strength hyperparameter for classification-weights `W`
* `alpha_d` -- sparsity parameter for `d`
* `alpha_W` -- sparsity parameter for `W`
* `Tmax` -- number of total iterations
* `record` -- how often during training to evaluate loss and accuracy
* `gamma` -- cadre-assignment sharpness hyperparameter

In this analysis, we're using a small `Tmax` value, but larger ones may be needed for more complex datasets.

Required arguments to SCM fit method:

* `Xtr` -- training feature values, in `np.array` format
* `Ytr` -- training labels, in `np.array` format

Optional arguments to SCM fit method:

* `Xva` -- validation feature values, in `np.array` format
* `Yva` -- validation labels, in `np.array` format
* `names` -- list or `pd.Index` of feature names
* `seed` -- RNG seed for parameter initalization SGD
* `store` -- whether or not to store copy of training data in SCM object, False by default
* `progress` -- whether or not to print diagnostics during training, False by default

If $N$ is the number of observations, and $P$ is the number of features, `Xtr` and `Xva` should be $N \times P$ arrays, and `Ytr` and `Yva` should be $N\times1$ arrays. If the labels are supplied as rank-1 arrays instead of rank-2 arrays, TensorFlow will automatically do some broadcasting that won't reflect what you want it do be doing.

The fit method doesn't automatically standardize data, so, if applicable, that should be performed prior to fitting

If `progress=True`, the printed diagnostics will be:

Iteration Number, Loss Value, Training Accuracy, Validation Accuracy (if applicable), Time

You can supply `Xva` and `Yva` to monitor for overfitting.

`alpha_d` and `alpha_W` should be between 0 and 1; if they are close to 1, then the parameters `d` and `W` will be more likely to be sparse.

The SCM optimization problem sometimes suffers from ill-conditioning. When this happens, it's best to change `gamma` or `lambda_d`. I've found that `gamma=10` works fairly well for datasets with tens of non-sparse features; as dimensionality increases, it may need to be decreased. Increasing `lambda_d` will also make estimated values of `d` smaller, which helps with conditioning.

In [ ]:
scores = {'M': [], 'lambda_d': [], 'lambda_W': [], 'accuracy': [], 'loss': []}
for M, l_d, l_W in product(scm_params['M'], scm_params['lambda_d'], scm_params['lambda_W']):
    print(M, l_d, l_W)
    for (tr, va) in kf.split(Xtr):
        ## split training data into training and validation sets
        x_tr, x_va, y_tr, y_va = Xtr[tr,:], Xtr[va,:], Ytr[tr,:], Ytr[va,:]
        ## standardize validation data with respect to training data and then standardize training data
        x_va = zmap(x_va, x_tr)
        x_tr = zscore(x_tr)
        ## initalize and fit SCM model object with current hyperparameters
        scm_mod = binaryCadreModel(M=M, lambda_d=l_d, lambda_W=l_W, Tmax=201, record=10, gamma=5.)
        scm_mod.fit(Xtr=x_tr, Ytr=y_tr, Xva=x_va, Yva=y_va, names=features, progress=False)
        ## update records
        scores['M'].append(M)
        scores['lambda_d'].append(l_d)
        scores['lambda_W'].append(l_W)
        scores['accuracy'].append(scm_mod.score(x_va, y_va))
        scores['loss'].append(scm_mod.loss[-1])
## transform scores in DataFrame for easy analysis
scores = pd.DataFrame(scores)

Identify best hyperparameter configuration

In [ ]:
best_hyperparameters = scores.groupby(['M','lambda_W','lambda_d']).mean().sort_values('accuracy', ascending=False)
best_hyperparameters

Estimate a model using all of the training data and the best hyperparameters

In [ ]:
best_M = best_hyperparameters['M'].values[0]
best_l_d = best_hyperparameters['lambda_d'].values[0]
best_l_W = best_hyperparameters['lambda_W'].values[0]

In [ ]:
scm_best = binaryCadreModel(M=best_M, lambda_d=best_l_d, lambda_W=best_l_W, Tmax=201, record=10, gamma=5.)
x_te = zmap(Xte, Xtr)
x_tr = zscore(Xtr)
scm_best.fit(Xtr=x_tr, Ytr=Ytr, Xva=x_te, Yva=Yte, names=features)

We can evaluate convergence by plotting loss and accuracy

In [ ]:
pd.DataFrame({'loss': scm_best.loss,
             'TrainingAccuracy': scm_best.accs,
            'TestingAccuracy': scm_best.accsVa}).plot()

We can look at the values of the classification weight parameter `W`. `W` is a set of $M$ length-$P$ column vectors. The value of the $p$th component in the $m$th column quantifies the association between the predicted label and that feature. As the value becomes more positive, the feature becomes more positively associated with the `+1` label.

When we look at a plot like this, it's often informative to see what features are used similarly between cadres and which are used differently. In the plot below, for example, `texture error` is associated with class `+1` in `w1` (orange) and associated with class `-1` in `w0` (blue). Also, `worst radius` has a much stronger association with class `-1` in `w1` than it does with `w2`.

In [ ]:
W_df = pd.DataFrame(scm_best.W, columns=['w0','w1'], index=scm_best.columns).reset_index().assign(baseline=0)
fig, ax = plt.subplots()
fig.set_size_inches(20, 5)
p = sns.lineplot(x='index', y='weight', hue='cadre', data=W_df.melt('index', var_name='cadre', value_name='weight'))
for item in p.get_xticklabels():
    item.set_rotation(45)

We can also look at the distributions of features by cadre. First we predict each training point's label and cadre.

In [ ]:
__, l_tr, __, m_tr = scm_best.predictFull(x_tr)
augmented_data = pd.DataFrame(x_tr, columns=scm_best.columns).assign(cadre=m_tr)

We print counts of every (cadre, true label, predicted label) combination. Cadre 0 primarily contains `+1` points, and cadre 1 primarily contains `-1` points.

In [ ]:
pd.DataFrame({'true_y': Ytr[:,0], 'pred_y': l_tr[:,0], 'cadre': m_tr}).groupby(['cadre', 'true_y', 'pred_y']).size()

We bind the features and cadre into a single `DataFrame` and find feature means, which we plot by cadre. They are very distinct.

In [ ]:
feature_means = augmented_data.groupby('cadre').mean().reset_index().melt(id_vars='cadre', var_name='feature', value_name='mean_value')
sns.lineplot(x='feature', y='mean_value', hue='cadre', data=feature_means)

The breastcancer dataset is fairly small, and training is quick. But for larger datasets, training will take longer, and it is advantageous to perform model selection by training in parallel. The main package you need for this is `joblib`, which implements parallelized `for` loops. (The common term is "embarassingly parallel".) We've also loading `multiprocessing`, but we only use it to detect how many cores we have access to.

First we see how many cores we have access to.

In [ ]:
cpu_count()

We can use some or all of these to speed up the process.

Notes:

* It's not always the best to use every core at once. Having to wait for each core's job to finish before moving on can produce delays. Also, TensorFlow will automatically parallelize some large matrix computations, I believe. So forcing each core to train a separate model can result in slower training times.
* It looks like Jupyter has access to 16 cores. Node-03 on the server has 48, although you have to run that through the command line.

Redefine hyperparameters and cross-validation setting. In practice, you'd want to use 10 or 20 folds.

In [ ]:
scm_params = {'M': np.array([2,3]), 'lambda_d': np.array([0.01, 0.1]), 'lambda_W': np.array([0.01, 0.1])}
kf = KFold(n_splits=3, random_state=1414)

First we define a function that trains a single model and returns its validation set accuracy.

In [ ]:
def scmCrossval(Xtr, Ytr, Xva, Yva, Tmax, M, a_W, l_W, a_d, l_d, gamma, features, fold):
    ## standardize validation data with respect to training data and then standardize training data
    x_va = zmap(Xva, Xtr)
    x_tr = zscore(Xtr)
    ## initalize and fit SCM model object with current hyperparameters
    scm_mod = binaryCadreModel(M=M, alpha_d=a_d, alpha_W=a_W, lambda_d=l_d, lambda_W=l_W, Tmax=Tmax, record=10, gamma=gamma)
    scm_mod.fit(Xtr=x_tr, Ytr=Ytr, names=features)
    
    ## extract final loss value
    loss = scm_mod.loss[-1]
    
    ## calculate training set accuracy
    tra_acc = scm_mod.score(x_tr, Ytr)
    
    ## calculate validation set accuracy
    val_acc = scm_mod.score(x_va, Yva)
    
    ## return everything as a list
    return fold, M, a_W, l_W, a_d, l_d, gamma, loss, tra_acc, val_acc

Now we invoke `joblib` to do the parallelized training. `joblib`'s `Parallel` function is the workhorse here. It's syntax is kind of verbose and confusing, unfortunately. First we describe the type of job we do, then we specify the function that is to be parallelized (wrapping it in `delayed`), and then we specify the parallelized functions arguments.

The parallelization backend we use is `"threading"`, as opposed to the default of `"multiprocessing"`. My experience is that `"threading"` works better when each parallelized function call (i.e., `scmCrossVal` call) is fairly memory-intensive. Setting `verbose=11` ensures that you are notified each time a job completes.

In [ ]:
n_jobs = 8
a_d = 0.9
a_W = 0.9
gamma = 5.
Tmax = 201

scores = (Parallel(n_jobs=8, backend='threading', verbose=11)(delayed(scmCrossval)
          (Xtr[tr,:], Ytr[tr,:], Xtr[va,:], Ytr[va,:], Tmax, M, a_W, l_W, a_d, l_d, gamma, features, fold) 
                        for (M, l_d, l_W, (fold, (tr, va))) in product(scm_params['M'], scm_params['lambda_d'], scm_params['lambda_W'], enumerate(kf.split(Xtr)))))

`Parallel` returns out cross-validation results as a list of tuples. So we need to reshape everything a `pd.DataFrame` for easier comparisons.

In [ ]:
results = {'fold': [], 'M': [], 'a_W': [], 'l_W': [], 'a_d': [], 'l_d': [], 'gamma': [], 'loss': [], 'training_acc': [], 
           'validation_acc': []}
for fold, M, a_W, l_W, a_d, l_d, gamma, loss, tra_acc, val_acc in scores:
    results['fold'].append(fold)
    results['M'].append(M)
    results['a_W'].append(a_W)
    results['l_W'].append(l_W)
    results['a_d'].append(a_d)
    results['l_d'].append(l_d)
    results['gamma'].append(gamma)
    results['loss'].append(loss)
    results['training_acc'].append(tra_acc)
    results['validation_acc'].append(val_acc)
results = pd.DataFrame(results)
results.drop('fold', axis=1).groupby(['M','a_W','l_W','a_d','l_d','gamma']).mean().sort_values('validation_acc', ascending=False)

Now we can choose optimal hyperparameters as before and train a final model.